<div style="border:1px solid black; padding:20px 20px;text-align: justify;text-justify: inter-word">
   <strong> Basics of Mobile Robotics: Project Report <br/> Autumn 2021 <br/><br/> Group members: </strong><br/><br/> Henry Papadatos <br/> Juliette Pelissier <br/> Eliott Sizey<br/> Aitana Waelbroeck-Boix
    
    
</div>

# Introduction

### Environment

### State Machine


Our program starts by initialising the environment by analysing the first frame sent by the camera through the vision module. It determines the starting position of the Thymio, the corners of the obstacles and the center of the goals. The optimal path is then defined from those information in the path planning module. <br>
The program then enters into the main loop which follows 4 steps:
- Vision: Determines the pose (position + angle) of the Thymio using the images sent by the camera
- Kalman filter: 
- Local avoidance:
- Motion control: 

PUT STATE MACHINE GRAPH HERE

# Vision

### Principle

The goal of the vision part of the project is to map the environment in which the robot will move (obstacles and goals) and to keep track of its pose. Each element of the environment has a different color which allows color differentiation and localisation. From a frame captured by the camera, the most efficient way of doing color detection is to transform this frame from the BGR domain to the HSV domain and apply a range color filter that turns pixels in the range to white and the other pixels to black. Once this is done, the shapes of the detected objects are also used as described in the following subsections. CAN ALSO TALK ABOUT BLUR, ERODE AND DILATE.

### HSV 

The HSV domain comprises three parameters that define a color: the hue, saturation and value. The hue 
Color detection

HSV principle

Obstacle expansion

Polygon 

# Global Navigation

Expliquer path planning

Peut montrer ce que ça donne avec le module séparé du reste 

# Local Navigation

local avoidance

memory

# Filtering

Kalman

Commenter la decision expanded kalman or not

initialisation matrices/tuning

# Motion Control

PID : mapping position sur 180 degrés 

# Conclusion